<a href="https://colab.research.google.com/github/DEVANG-2021/ZeroShield---AI-Powered-Adversarial-Defense-for-Zero-Trust-Security/blob/main/Cyber_Secturity_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import foolbox as fb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Load CICIDS2017 dataset
def load_dataset(file_path):
    df = pd.read_csv(file_path)
    df.fillna(0, inplace=True)  # Handle missing values
    return df[:1000]  # Use only first 5000 data points

# Normalize dataset
def normalize_data(df):
    scaler = MinMaxScaler()
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    df = df[numerical_cols]  # Drop non-numeric columns
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(df.mean(), inplace=True)
    df = df.astype(np.float32)
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    return df

# Define GAN for adversarial attack generation
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Train GAN
def train_gan(generator, discriminator, data, epochs=500, batch_size=128, lr=0.0002):
    criterion = nn.BCELoss()
    optimizer_g = optim.AdamW(generator.parameters(), lr=lr)
    optimizer_d = optim.AdamW(discriminator.parameters(), lr=lr)
    dataloader = DataLoader(TensorDataset(data), batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        total_samples = 0
        for real_samples in dataloader:
            real_samples = real_samples[0]
            batch_size = real_samples.size(0)
            total_samples += batch_size

            # Generate fake samples
            noise = torch.randn(batch_size, real_samples.shape[1])
            fake_samples = generator(noise)

            # Train Discriminator
            real_labels = torch.ones(batch_size, 1)
            fake_labels = torch.zeros(batch_size, 1)
            optimizer_d.zero_grad()
            loss_real = criterion(discriminator(real_samples), real_labels)
            loss_fake = criterion(discriminator(fake_samples.detach()), fake_labels)
            loss_d = loss_real + loss_fake
            loss_d.backward()
            optimizer_d.step()

            # Train Generator
            optimizer_g.zero_grad()
            loss_g = criterion(discriminator(fake_samples), real_labels)
            loss_g.backward()
            optimizer_g.step()

        print(f"Epoch [{epoch+1}/{epochs}] - Total Samples Processed: {total_samples}")

    return generator

# Example usage
dataset_path = "/content/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"  # Replace with actual path
df = load_dataset(dataset_path)
df = normalize_data(df)
print(df.dtypes)
print(df.head())
print("Generator Start Running...")
generator = Generator(input_dim=df.shape[1], output_dim=df.shape[1])
print("Generator Stop Running...")

print("Discriminator Start Running...")
discriminator = Discriminator(input_dim=df.shape[1])
print("Discriminator Stop Running...")

trained_generator = train_gan(generator, discriminator, torch.tensor(df.values, dtype=torch.float32))
print("GAN trained successfully!")

import gymnasium as gym  # Use Gymnasium instead of Gym
import stable_baselines3 as sb3
from stable_baselines3.common.vec_env import DummyVecEnv

# Reinforcement Learning for Adaptive Defense
def train_rl_agent(env_name="CartPole-v1", timesteps=10000):
    # Create environment and wrap it with DummyVecEnv
    env = gym.make(env_name)  # Gymnasium environment
    env = DummyVecEnv([lambda: env])  # Wrap the environment in a DummyVecEnv

    # Reset the environment and handle dictionary structure for gymnasium
    reset_output = env.reset()  # Reset and get the output (this will be a dictionary)
    obs = reset_output[0]  # First element is the observation (from DummyVecEnv)
    info = reset_output[1] if isinstance(reset_output, tuple) else {}  # Extract info if available

    model = sb3.PPO("MlpPolicy", env, verbose=1)
    model.learn(total_timesteps=timesteps)
    return model

# Train Reinforcement Learning Agent
rl_model = train_rl_agent()
print("RL Agent trained successfully!")

/usr/local/lib/python3.11/dist-packages/foolbox/attacks/blur.py:3: DeprecationWarning: Please import `gaussian_filter` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import gaussian_filter
<ipython-input-10-359f0a612773>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace([np.inf, -np.inf], np.nan, inplace=True)
<ipython-input-10-359f0a612773>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(df.mean(), inplace=True)


 Destination Port              float32
 Flow Duration                 float32
 Total Fwd Packets             float32
 Total Backward Packets        float32
Total Length of Fwd Packets    float32
                                ...   
 Active Min                    float32
Idle Mean                      float32
 Idle Std                      float32
 Idle Max                      float32
 Idle Min                      float32
Length: 78, dtype: object
    Destination Port   Flow Duration   Total Fwd Packets  \
0           0.900071    2.500995e-08            0.000621   
1           0.903172    9.086947e-07            0.000000   
2           0.903189    4.335057e-07            0.000000   
3           0.758456    2.834461e-07            0.000000   
4           0.900038    2.500995e-08            0.000621   

    Total Backward Packets  Total Length of Fwd Packets  \
0                 0.000000                     0.000404   
1                 0.000517                     0.000202   
2      

# New Section

In [ ]:
pip install foolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.0 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym  # Use Gymnasium instead of Gym
import stable_baselines3 as sb3
from stable_baselines3.common.vec_env import DummyVecEnv
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim

# After normalizing the dataset (after 'normalize_data(df)' call):
def create_graph_from_data(df):
    num_nodes = len(df)
    edges = np.random.randint(0, num_nodes, size=(2, 100))  # Adjust as per real connection logic
    edge_index = torch.tensor(edges, dtype=torch.long)

    features = torch.tensor(df.values, dtype=torch.float)

    data = Data(x=features, edge_index=edge_index)
    return data

class GATModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GATModel, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=8, dropout=0.6)
        self.conv2 = GATConv(hidden_channels * 8, out_channels, heads=1, dropout=0.6)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.nn.functional.elu(x)
        x = self.conv2(x, edge_index)
        return x

def train_gat_model(data, epochs=100, lr=0.001):
    model = GATModel(in_channels=data.num_node_features, hidden_channels=64, out_channels=1)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(data.x, data.edge_index)  # Forward pass
        loss = criterion(output, torch.zeros_like(output))  # Adjust target for anomaly detection
        loss.backward()
        optimizer.step()

        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {loss.item()}")

    return model

def evaluate_model(model, data):
    model.eval()
    output = model(data.x, data.edge_index)
    anomalous_nodes = output > 0.5  # Anomaly detection threshold
    print(f"Anomalous Nodes: {anomalous_nodes.nonzero()}")

class FLModel(nn.Module):
    def __init__(self, input_dim):
        super(FLModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Binary classification (for simplicity)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

class Client(fl.client.NumPyClient):
    def __init__(self, model, data, targets):
        self.model = model
        self.data = data
        self.targets = targets
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.criterion = nn.BCELoss()

    def get_parameters(self):
        return [val.cpu().numpy() for val in self.model.parameters()]

    def set_parameters(self, parameters):
        params = zip(self.model.parameters(), parameters)
        for param, value in params:
            param.data = torch.tensor(value, dtype=torch.float32)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.optimizer.zero_grad()
        output = self.model(self.data)
        loss = self.criterion(output.squeeze(), self.targets.float())
        loss.backward()
        self.optimizer.step()
        return self.get_parameters(), len(self.data), loss.item()

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        with torch.no_grad():
            output = self.model(self.data)
            loss = self.criterion(output.squeeze(), self.targets.float())
        return loss.item(), len(self.data), {"accuracy": (output.round() == self.targets).float().mean().item()}

def split_dataset(df, num_clients=5):
    # Split the dataframe into smaller chunks (simulate multiple nodes)
    chunk_size = len(df) // num_clients
    clients_data = []
    clients_targets = []

    for i in range(num_clients):
        chunk = df.iloc[i * chunk_size: (i + 1) * chunk_size]
        data = torch.tensor(chunk.values, dtype=torch.float32)
        targets = torch.tensor(np.random.randint(0, 2, size=(len(chunk),)), dtype=torch.long)  # Random binary target
        clients_data.append(data)
        clients_targets.append(targets)

    return clients_data, clients_targets

from flwr.server.strategy import FedAvg
from flwr.server import start_server
from flwr.server.server import ServerConfig

def start_federated_learning():
    # Initialize the global model
    # global_model = FLModel(input_dim=df.shape[1])

    # # Split dataset for clients (representing power grid nodes)
    # clients_data, clients_targets = split_dataset(df, num_clients=5)

    # # Create clients
    # clients = []
    # for i in range(len(clients_data)):
    #     client = Client(global_model, clients_data[i], clients_targets[i])
    #     clients.append(client)

    # # Define the federated learning strategy
    # strategy = fl.server.strategy.FedAvg(
    #     fraction_fit=0.5,  # Fraction of clients to use for training
    #     min_available_clients=2  # Ensure at least 2 clients are available for training
    # )

    # Start the Flower server for federated learning
    strategy = FedAvg(
      fraction_fit=0.5,  # Fraction of clients used for training
      min_fit_clients=2,  # Minimum number of clients needed for training
      min_available_clients=2  # Ensure at least 2 clients are available
    )

     # Create ServerConfig object
    config = ServerConfig(num_rounds=1)

    # Start the FL server using ServerConfig
    start_server(server_address="0.0.0.0:8085", config=config)


# Reinforcement Learning for Adaptive Defense
def train_rl_agent(env_name="CartPole-v1", timesteps=10000):
    # Create environment and wrap it with DummyVecEnv
    env = gym.make(env_name)  # Gymnasium environment
    env = DummyVecEnv([lambda: env])  # Wrap the environment in a DummyVecEnv

    # Reset the environment and handle dictionary structure for gymnasium
    reset_output = env.reset()  # Reset and get the output (this will be a dictionary)
    obs = reset_output[0]  # First element is the observation (from DummyVecEnv)
    info = reset_output[1] if isinstance(reset_output, tuple) else {}  # Extract info if available

    model = sb3.PPO("MlpPolicy", env, verbose=1)
    model.learn(total_timesteps=timesteps)
    return model

# Step 3: Create Graph from Data (Graph Representation)
data = create_graph_from_data(df)

# Step 4: Train GAT Model for Anomaly Detection
gat_model = train_gat_model(data)

# Step 5: Evaluate GAT Model for Anomalies
evaluate_model(gat_model, data)


Epoch [1/100] - Loss: 0.6654284596443176
Epoch [2/100] - Loss: 0.6031944751739502
Epoch [3/100] - Loss: 0.5626355409622192
Epoch [4/100] - Loss: 0.5301454067230225
Epoch [5/100] - Loss: 0.5158560872077942
Epoch [6/100] - Loss: 0.49301624298095703
Epoch [7/100] - Loss: 0.4675811231136322
Epoch [8/100] - Loss: 0.46819007396698
Epoch [9/100] - Loss: 0.4452321529388428
Epoch [10/100] - Loss: 0.4394039213657379
Epoch [11/100] - Loss: 0.448520302772522
Epoch [12/100] - Loss: 0.43978312611579895
Epoch [13/100] - Loss: 0.4143456816673279
Epoch [14/100] - Loss: 0.438574880361557
Epoch [15/100] - Loss: 0.42987436056137085
Epoch [16/100] - Loss: 0.403044730424881
Epoch [17/100] - Loss: 0.41572850942611694
Epoch [18/100] - Loss: 0.4182533621788025
Epoch [19/100] - Loss: 0.40243732929229736
Epoch [20/100] - Loss: 0.41281768679618835
Epoch [21/100] - Loss: 0.3998563885688782
Epoch [22/100] - Loss: 0.4058057963848114
Epoch [23/100] - Loss: 0.3900117874145508
Epoch [24/100] - Loss: 0.39360061287879944

In [ ]:
# Step 6: Start Federated Learning (FL)
# start_federated_learning()  # Federated Learning Setup
# print("Federated Learning done successfully!")
from threading import Thread
Thread(target=start_federated_learning).start()

# Step 7: Proceed to RL Agent Training (you can move ahead with RL as before)
# rl_model = train_rl_agent()
# print("RL Agent trained successfully!")

	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


In [ ]:
!pip install pyngrok flwr torch --quiet
from pyngrok import ngrok

# Start ngrok tunnel
public_url = ngrok.connect(8080, "tcp")
print(f"Server accessible at: {public_url}")

# Modified server code
def start_server():
    strategy = fl.server.strategy.FedAvg(
        min_fit_clients=2,
        min_available_clients=2
    )
    fl.server.start_server(
        server_address="0.0.0.0:8080",
        config={"num_rounds": 3},
        strategy=strategy
    )

# Run in background
import threading
server_thread = threading.Thread(target=start_server)
server_thread.start()

ERROR:pyngrok.process.ngrok:t=2025-03-13T00:52:20+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-03-13T00:52:20+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-03-13T00:52:20+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
# Server (run first)
def start_server():
    fl.server.start_server(
        server_address="0.0.0.0:8080",
        config={"num_rounds": 3},
        strategy=fl.server.strategy.FedAvg(
            min_fit_clients=2,
            min_available_clients=2
        )
    )

# Client (run in separate cells)
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, trainloader):
        self.model = model
        self.trainloader = trainloader

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def fit(self, parameters, config):
        self.model.set_parameters(parameters)
        # Add training logic
        return self.get_parameters(config), len(self.trainloader), {}

# Client connection (run in separate cells)
def client_fn(cid: str) -> fl.client.Client:
    model = Net()  # Your model class
    trainloader = ...  # Your data
    return FlowerClient(model, trainloader).to_client()

# Start multiple clients
for i in range(2):
    client_thread = threading.Thread(target=client_fn, args=(str(i),))
    client_thread.start()

Exception in thread Thread-65 (client_fn):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-56-d164ee370dd6>", line 30, in client_fn
NameError: name 'Net' is not defined. Did you mean: 'set'?
Exception in thread Thread-66 (client_fn):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-56-d164ee370dd6>", line 30, in client_fn
NameError: name 'Net' is not defined. Did you mean: 'set'?


In [ ]:
pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [ ]:
pip install shimmy


In [ ]:
pip install gym

In [ ]:
pip install gymnasium

In [ ]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.6 MB/s eta 0:00:00


In [ ]:
pip install flwr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.1/532.1 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.15.2
    Uninstalling typer-0.15.2:
      Successfully uninstalled typer-0.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.2 which is incompa

In [ ]:
pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 6.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nv